In [1]:
# 读取../visualization/assets/data/matrix/new_community_evolution.csv

import os
import pandas as pd
import json
import numpy as np

handle_df = pd.read_csv("matrix/new_community_evolution.csv")
# 与上下文无关
# handle坐标需要重新计算，最大半径320
import numpy as np

RAD = np.pi / 180

# 添加处理NaN的函数
def handle_nan(obj):
    """处理JSON序列化时的NaN值，将其替换为null"""
    if isinstance(obj, dict):
        return {k: handle_nan(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [handle_nan(i) for i in obj]
    elif isinstance(obj, float) and np.isnan(obj):
        return None
    else:
        return obj

def polarToCartesian(asrank, attr, radius=350):
    """
    计算xy坐标的实现函数
    :param asrank: 需要处理的asrank数据，保证为DataFrame格式，且必须要拥有longitude、attr字段
    :param attr: 指定as的某个属性，这里使用cone
    :param radius: 节点的最大极半径
    :return:
    """
    # 映射值域cone的min-max  映射到   0-350
    min_cone = asrank[attr].min()
    max_cone = asrank[attr].max()

    # cone原值域太大，进行值域缩放，用来后面求半径
    max_Domain = 1 - np.log((min_cone + 1) / (max_cone + 1))

    # cone的log处理
    asrank['cone_map'] = asrank[attr].apply(lambda x: (1 - np.log((x + 1) / (max_cone + 1))))

    # log处理后，对cone进行一个0-350区间映射
    asrank['r'] = asrank['cone_map'].apply(lambda x: (((x - 1) / (max_Domain - 1)) * radius))

    # 得到xy坐标 x=初始位置+rcosθ
    x = 1900 / 2 + asrank['r'] * np.cos(asrank['longitude'] * RAD)
    y = 1000 / 2 - asrank['r'] * np.sin(asrank['longitude'] * RAD)

    asrank['x'] = x
    asrank['y'] = y
    # 将rank 的r字段全部数值变为绝对值
    asrank.sort_values(by=attr, ascending=False, inplace=True)
    asrank.drop_duplicates(subset=['id', attr], keep='last', inplace=True)

    return asrank


# for date in range(202404, 202411):
#     cord_rank = pd.read_csv(f'./{date}/handle/rank{date}.csv')
#     cord_rank1 = pd.read_csv(f'./{date}/rank{date}.csv')
#     cord_rank = polarToCartesian(cord_rank, 'cone', 320)
#     cord_rank1 = polarToCartesian(cord_rank1, 'cone', 320)
#     cord_rank.fillna(0, inplace=True)
#     cord_rank1.fillna(0, inplace=True)
#     cord_rank.to_csv(f'./{date}/handle/rank{date}.csv', index=False)
#     cord_rank1.to_csv(f'./{date}/rank{date}.csv', index=False)


base_dir = './'

# 初始化 rankjson 和 reljson
rankjson = {}
reljson = {}

# 获取 handle_df 中每一列的所有唯一值
for col in handle_df.columns:
    unique_values = handle_df[col].unique()  # 获取该列的所有唯一值
    rankjson[col] = {}  # 创建该列对应的 rankjson 结构
    reljson[col] = {}  # 创建该列对应的 reljson 结构

    # 遍历该列的所有唯一值
    for value in unique_values:
        # 将 value 转换为字符串，以确保 JSON 兼容
        value_str = str(value)

        # 根据列名获取对应的 rank 和 rel 文件路径
        rank_path = os.path.join(base_dir, f'{col}/handle/rank{col}.csv')
        rel_path = os.path.join(base_dir, f'{col}/handle/rel{col}.csv')

        # 读取 rank 和 rel 文件
        temp_rank = pd.read_csv(rank_path)
        temp_rel = pd.read_csv(rel_path)

        # 根据条件筛选数据
        filtered_rank = temp_rank[temp_rank['community'] == value]
        filtered_rel = temp_rel[temp_rel['type'] == str(value)]
        filtered_rank = polarToCartesian(filtered_rank, 'cone', 320)

        if len(filtered_rank) != 0 and len(filtered_rel) != 0:
            filtered_rel['cone'] = filtered_rel.apply(
                lambda row: max(
                    filtered_rank.loc[filtered_rank['id'] == row['source'], 'cone'].iloc[0],  # 获取 source 对应的 cone 值
                    filtered_rank.loc[filtered_rank['id'] == row['target'], 'cone'].iloc[0]  # 获取 target 对应的 cone 值
                ),
                axis=1
            )

        os.makedirs(f'./local_core/{col}/raw', exist_ok=True)
        # 将筛选结果存储到对应的 rankjson 和 reljson 中
        # 替换NaN为None，以便JSON序列化
        rankjson[col][value_str] = handle_nan(filtered_rank.replace({np.nan: None}).to_dict(orient='records'))
        reljson[col][value_str] = handle_nan(filtered_rel.replace({np.nan: None}).to_dict(orient='records'))

    with open(f'./local_core/{col}/raw/rank.json', 'w', encoding='utf-8') as rank_file:
        json.dump(rankjson, rank_file, ensure_ascii=False, indent=4)

    with open(f'./local_core/{col}/raw/rel.json', 'w', encoding='utf-8') as rel_file:
        json.dump(reljson, rel_file, ensure_ascii=False, indent=4)
    print(f"Saved data for {col}")


# 节点聚合，根据国家进行聚合，然后相应的连接关系也进行聚合，并导出
def process_rank_and_rel(t1, t2):
    t1.fillna(0, inplace=True)
    t1_agg = t1.loc[t1.groupby('country')['cone'].idxmax()]

    t1_agg['cone'] = t1_agg['country'].apply(
        lambda country: t1[t1['country'] == country]['cone'].sum()
    )
    t1_agg = polarToCartesian(t1_agg, 'cone', 320)

    id_mapping = {}
    for idx, row in t1.iterrows():
        country = row['country']
        key = row['id']
        value = t1_agg[t1_agg['country'] == country]['id'].values[0]
        id_mapping[key] = value

    t2['source'] = t2['source'].map(id_mapping).fillna(t2['source'])
    t2['target'] = t2['target'].map(id_mapping).fillna(t2['target'])
    t2['source'] = t2['source'].astype(int)
    t2['target'] = t2['target'].astype(int)
    t2 = t2[t2['source'] != t2['target']]
    t2['source_target'] = t2.apply(lambda row: tuple(sorted([row['source'], row['target']])), axis=1)
    t2['reverse_relation'] = t2.apply(lambda row: tuple(sorted([row['target'], row['source']])), axis=1)
    t2 = t2.drop_duplicates(subset=['source_target'])
    t2 = t2.drop_duplicates(subset=['reverse_relation'])
    t2 = t2.drop_duplicates(subset=['source', 'target'])
    t2 = t2.drop(columns=['source_target'])
    t2 = t2.drop(columns=['reverse_relation'])

    if len(t1_agg) != 0 and len(t2) != 0:
        t2['cone'] = t2.apply(
            lambda row: max(
                t1_agg.loc[t1_agg['id'] == row['source'], 'cone'].iloc[0],  # 获取 source 对应的 cone 值
                t1_agg.loc[t1_agg['id'] == row['target'], 'cone'].iloc[0]  # 获取 target 对应的 cone 值
            ),
            axis=1
        )

    return t1_agg, t2


base_dir = './'

# 初始化 rankjson 和 reljson
rankjson = {}
reljson = {}

# 获取 handle_df 中每一列的所有唯一值
for col in handle_df.columns:
    unique_values = handle_df[col].unique()  # 获取该列的所有唯一值
    rankjson[col] = {}
    reljson[col] = {}
    os.makedirs(f'./local_core/{col}/agg', exist_ok=True)
    # 遍历该列的所有唯一值
    for value in unique_values:
        # 将 value 转换为字符串，以确保 JSON 兼容
        value_str = str(value)

        # 根据列名获取对应的 rank 和 rel 文件路径
        rank_path = os.path.join(base_dir, f'{col}/handle/rank{col}.csv')
        rel_path = os.path.join(base_dir, f'{col}/handle/rel{col}.csv')

        # 读取 rank 和 rel 文件
        temp_rank = pd.read_csv(rank_path)
        temp_rel = pd.read_csv(rel_path)

        # 根据条件筛选数据
        filtered_rank = temp_rank[temp_rank['community'] == value]
        filtered_rel = temp_rel[temp_rel['type'] == str(value)]
        t1, t2 = process_rank_and_rel(filtered_rank, filtered_rel)

        if len(t1) == 1:
            t1.loc[t1.index[0], 'x'] = 1900 / 2
            t1.loc[t1.index[0], 'y'] = 1000 / 2
            t1.loc[t1.index[0], 'r'] = 0
            
        # 将筛选结果存储到对应的 rankjson 和 reljson 中，处理NaN值
        rankjson[col][value_str] = handle_nan(t1.replace({np.nan: None}).to_dict(orient='records'))
        reljson[col][value_str] = handle_nan(t2.replace({np.nan: None}).to_dict(orient='records'))

    with open(f'./local_core/{col}/agg/rank.json', 'w', encoding='utf-8') as rank_file:
        json.dump(rankjson, rank_file, ensure_ascii=False, indent=4)

    with open(f'./local_core/{col}/agg/rel.json', 'w', encoding='utf-8') as rel_file:
        json.dump(reljson, rel_file, ensure_ascii=False, indent=4)
    print(f"Saved data for {col}")

### 处理fdeb的逻辑
from common.utils.As_rank_rel import fdeb_format

base_dir = './'

# 初始化 rankjson 和 reljson
rankjson = {}
reljson = {}

# 获取 handle_df 中每一列的所有唯一值
for col in handle_df.columns:
    unique_values = handle_df[col].unique()  # 获取该列的所有唯一值
    rankjson[col] = {}
    reljson[col] = {}
    os.makedirs(f'./local_core/{col}/fdeb', exist_ok=True)
    # 遍历该列的所有唯一值
    for value in unique_values:
        # 将 value 转换为字符串，以确保 JSON 兼容
        value_str = str(value)

        # 根据列名获取对应的 rank 和 rel 文件路径
        rank_path = os.path.join(base_dir, f'{col}/handle/rank{col}.csv')
        rel_path = os.path.join(base_dir, f'{col}/handle/rel{col}.csv')

        # 读取 rank 和 rel 文件
        temp_rank = pd.read_csv(rank_path)
        temp_rel = pd.read_csv(rel_path)

        # 根据条件筛选数据
        filtered_rank = temp_rank[temp_rank['community'] == value]
        filtered_rel = temp_rel[temp_rel['type'] == str(value)]
        t1, t2 = process_rank_and_rel(filtered_rank, filtered_rel)

        if len(t1) == 1:
            t1.loc[t1.index[0], 'x'] = 1900 / 2
            t1.loc[t1.index[0], 'y'] = 1000 / 2
            t1.loc[t1.index[0], 'r'] = 0

        r1, r2 = fdeb_format(t1, t2)
        # 确保处理fdeb格式数据中的NaN值
        rankjson[col][value_str] = handle_nan(r1)
        reljson[col][value_str] = handle_nan(r2)
        
    with open(f'./local_core/{col}/fdeb/rank.json', 'w', encoding='utf-8') as rank_file:
        json.dump(rankjson, rank_file, ensure_ascii=False, indent=4)

    with open(f'./local_core/{col}/fdeb/rel.json', 'w', encoding='utf-8') as rel_file:
        json.dump(reljson, rel_file, ensure_ascii=False, indent=4)
    print(f"Saved data for {col}")


/tmp/ipykernel_20749/3351693881.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asrank['cone_map'] = asrank[attr].apply(lambda x: (1 - np.log((x + 1) / (max_cone + 1))))
/tmp/ipykernel_20749/3351693881.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asrank['r'] = asrank['cone_map'].apply(lambda x: (((x - 1) / (max_Domain - 1)) * radius))
/tmp/ipykernel_20749/3351693881.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

Saved data for 202401


/tmp/ipykernel_20749/3351693881.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asrank['cone_map'] = asrank[attr].apply(lambda x: (1 - np.log((x + 1) / (max_cone + 1))))
/tmp/ipykernel_20749/3351693881.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asrank['r'] = asrank['cone_map'].apply(lambda x: (((x - 1) / (max_Domain - 1)) * radius))
/tmp/ipykernel_20749/3351693881.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

Saved data for 202402


/tmp/ipykernel_20749/3351693881.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asrank['cone_map'] = asrank[attr].apply(lambda x: (1 - np.log((x + 1) / (max_cone + 1))))
/tmp/ipykernel_20749/3351693881.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asrank['r'] = asrank['cone_map'].apply(lambda x: (((x - 1) / (max_Domain - 1)) * radius))
/tmp/ipykernel_20749/3351693881.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

...

Saved data for 202408


/tmp/ipykernel_20749/3351693881.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.fillna(0, inplace=True)
/tmp/ipykernel_20749/3351693881.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['source'] = t2['source'].map(id_mapping).fillna(t2['source'])
/tmp/ipykernel_20749/3351693881.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

Saved data for 202409


/tmp/ipykernel_20749/3351693881.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.fillna(0, inplace=True)
/tmp/ipykernel_20749/3351693881.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['source'] = t2['source'].map(id_mapping).fillna(t2['source'])
/tmp/ipykernel_20749/3351693881.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

Saved data for 202410
